In [ ]:
%pip install scikit-learn

import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
def detect_coins(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (9, 9), 2)

    # Detect circles
    circles = cv2.HoughCircles(
        gray, 
        cv2.HOUGH_GRADIENT, 
        dp=1.2, 
        minDist=100, 
        param1=100, 
        param2=70, 
        minRadius=40, 
        maxRadius=120
    )

    annotated = image.copy()
    total = 0
    counts = {1:0, 5:0, 10:0, 20:0}

    if circles is not None:
        circles = np.uint16(np.around(circles[0, :]))
        circles = sorted(circles, key=lambda c: c[2], reverse=True)
        detected = []

        for (x, y, r) in circles:
            if any(np.hypot(x - dx, y - dy) < 0.8 * r for dx, dy, dr in detected):
                continue
            detected.append((x, y, r))

            x1, y1 = max(0, x-r), max(0, y-r)
            x2, y2 = min(image.shape[1], x+r), min(image.shape[0], y+r)
            coin_roi = image[y1:y2, x1:x2]

            val = None  

            hsv = cv2.cvtColor(coin_roi, cv2.COLOR_BGR2HSV)

            lower_gold = np.array([10, 40, 50])   
            upper_gold = np.array([40, 255, 255]) 

            mask_gold = cv2.inRange(hsv, lower_gold, upper_gold)

            mask = np.zeros(mask_gold.shape, dtype="uint8")
            cv2.circle(mask, (mask.shape[1]//2, mask.shape[0]//2), int(0.8*mask.shape[0]//2), 255, thickness=15)
            gold_ring = cv2.bitwise_and(mask_gold, mask)

            gold_ratio = np.sum(gold_ring > 0) / (coin_roi.shape[0] * coin_roi.shape[1])

            if gold_ratio > 0.02:  
                val = 20
            else:
                if r < 55:
                    val = 1
                elif r < 70:
                    val = 5
                else:
                    val = 10

            counts[val] += 1
            total += val

            cv2.circle(annotated, (x, y), r, (0, 255, 0), 3)
            cv2.putText(annotated, f"₱{val}", (x-30, y), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    cv2.putText(annotated, f"Total: ₱{total}", (20, 40), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

    return annotated, counts, total

In [4]:
class CoinApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Philippine Peso Coin Detector")

        self.video_label = tk.Label(root)
        self.video_label.pack()

        self.start_button = tk.Button(root, text="Start Camera", command=self.start_camera)
        self.start_button.pack(side=tk.LEFT, padx=5, pady=5)

        self.stop_button = tk.Button(root, text="Stop Camera", command=self.stop_camera)
        self.stop_button.pack(side=tk.LEFT, padx=5, pady=5)

        self.snapshot_button = tk.Button(root, text="Save Snapshot", command=self.save_snapshot)
        self.snapshot_button.pack(side=tk.LEFT, padx=5, pady=5)

        self.cap = None
        self.frame = None
        self.running = False

    def start_camera(self):
        if not self.running:
            self.cap = cv2.VideoCapture(0)  # webcam index
            self.running = True
            self.update_frame()

    def stop_camera(self):
        if self.running:
            self.running = False
            self.cap.release()

    def save_snapshot(self):
        if self.frame is not None:
            cv2.imwrite("snapshot.png", cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB))
            print("Snapshot saved as snapshot.png")

    def update_frame(self):
        if self.running:
            ret, frame = self.cap.read()
            if ret:
                frame, counts, total = detect_coins(frame)
                self.frame = frame
                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(img)
                imgtk = ImageTk.PhotoImage(image=img)
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)
            self.root.after(10, self.update_frame)

In [6]:
root = tk.Tk()
app = CoinApp(root)
root.mainloop()